In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets import load_dataset

In [3]:
msmarco = load_dataset("ms_marco", "v1.1")

### Collection

In [16]:
import tqdm

N = 150000

all_passages = set()
for split, ds in msmarco.items():
    for row in tqdm.tqdm(ds["passages"]):
        for passage in row["passage_text"]:
            all_passages.add(passage)
        if len(all_passages) >= N:
            break

all_passages = sorted(all_passages)
passage_map = {txt: i for i, txt in enumerate(all_passages)}

  0%|          | 0/9650 [00:00<?, ?it/s]


In [17]:
import pandas as pd

collection = pd.DataFrame(all_passages, columns=["passage"])
collection["pid"] = list(range(len(all_passages)))
collection = collection[["pid", "passage"]]
collection.to_csv("../data/msmarco/collection.tsv", sep="\t", index=False, header=False)

### Queries

In [18]:
import jsonlines

queries_map = {}
for split, ds in msmarco.items():
    queries = []
    for i, row in enumerate(tqdm.tqdm(ds)):
        queries.append({"qid": i, "question": row["query"]})
        queries_map[row["query_id"]] = i

    print(len(queries))

    if split == "validation":
        split = "val"

    with jsonlines.open(f"../data/msmarco/queries_{split}.json", "w") as fh:
        for row in queries:
            fh.write(row)

  0%|          | 0/10047 [00:00<?, ?it/s]

100%|██████████| 10047/10047 [00:00<00:00, 14337.02it/s]


10047


100%|██████████| 82326/82326 [00:05<00:00, 15197.83it/s]


82326


100%|██████████| 9650/9650 [00:00<00:00, 15230.17it/s]

9650


### Triples

In [19]:
import numpy as np
import tqdm

np.random.seed(1234)


nway = 64
pids_shuffled = np.random.permutation(len(all_passages))
p_ptr = 0
skipped = {"qid": 0, "pid": 0}
for split, ds in msmarco.items():
    triples = []

    for row in tqdm.tqdm(ds):

        if not row["query_id"] in queries_map:
            if split == "train":
                skipped["qid"] += 1
            continue

        passages = row["passages"]
        is_selected = np.array(passages["is_selected"], dtype=bool)
        if not is_selected.sum():
            continue
        else:
            positive_pid = passage_map.get(passages["passage_text"][np.where(is_selected)[0][0]], None)
            if positive_pid is None:
                if split == "train":
                    skipped["pid"] += 1
                continue
        negative_pids = [passage_map.get(passages["passage_text"][i], None) for i in np.where(is_selected == 0)[0]]
        negative_pids = [pid for pid in negative_pids if pid is not None]
        while len(negative_pids) < nway:
            n_to_add = nway - len(negative_pids)
            if p_ptr + n_to_add >= len(pids_shuffled):
                pids_shuffled = np.random.permutation(len(all_passages))
                p_ptr = 0
            pids_to_add = pids_shuffled[p_ptr : p_ptr + n_to_add]
            if not positive_pid in pids_to_add:
                negative_pids += pids_to_add.tolist()
            p_ptr += n_to_add

        triples.append([queries_map[row["query_id"]], [positive_pid, 1.0], *[[pid, 0.0] for pid in negative_pids]])

    if split == "validation":
        split = "val"

    print(len(triples))

    with jsonlines.open(f"../data/msmarco/triples_{split}.json", "w") as fh:
        for line in triples:
            fh.write(line)

  0%|          | 0/10047 [00:00<?, ?it/s]

100%|██████████| 10047/10047 [00:01<00:00, 6785.86it/s]


9706


100%|██████████| 82326/82326 [00:07<00:00, 11670.07it/s]


11956


100%|██████████| 9650/9650 [00:00<00:00, 13090.48it/s]

454
